# Gender Classification CNN (VGG16, VGG-Face Transfer Learning)

## Import Dependencies

In [1]:
import os
import numpy as np
from numpy import array
from numpy import argmax
import imageio
import matplotlib.pyplot as plt
import pandas as pd
import glob
import keras
import time
from keras import backend as K
from keras.layers.core import Dense
from keras.layers import Convolution2D, MaxPooling2D, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.callbacks import CSVLogger, EarlyStopping, TensorBoard, ModelCheckpoint 
import matplotlib.pyplot as plt
from scipy.misc import imread, imresize

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras import backend as K
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

%matplotlib inline

Using TensorFlow backend.


## Preprocessing (Data Augmentation Approach)

In [2]:
#Read labels CSV
image_labels = pd.read_csv('train_target.csv')

In [5]:
#prepare data for flow_from_dir
from shutil import copyfile

#Split training and testing 80% and 20% respectively
data_size = len(image_labels.index)
train_size = data_size - (data_size * 0.2)
count = 0

#Iterate through labels to create directories (data/train) and (data/validation) separated by classes
for index, row in image_labels.iterrows():
    name = row['Id']
    gender = str(row['Gender']) 

    #For the first 80%, copy to data/train
    if count<train_size:
        
        #Check if dir exists then copy the image to the directory, otherwise create the directory and copy the image over.
        if os.path.isdir('data/train/' + gender):
            copyfile('train/'+name, 'data/train/'+gender+'/'+name)
        else:
            os.makedirs('data/train/'+gender)
            copyfile('train/'+name, 'data/train/'+gender+'/'+name)

        count = count + 1

    #For the last 20% of data, copy to validation with same logic as previous.
    else:
        if os.path.isdir('data/validation/' + gender):
            copyfile('train/'+name, 'data/validation/'+gender+'/'+name)
        else:
            os.makedirs('data/validation/'+gender)
            copyfile('train/'+name, 'data/validation/'+gender+'/'+name)
    

### Helper Function: Plot images

In [3]:
def plotImage(image):
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(image)
    axarr[0].grid()
    axarr[0].set_title('Image')

## Preprocessing (Standard Fitting Approach/No augmentation)

In [12]:
X_train = []
Y_train = []
for index, row in image_labels.iterrows():
    image = imread('train/' + row[0], mode='RGB')
    image = imresize(image, (244,244,3))
    X_train.append(np.array(image))
    Y_train.append(row[1])

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  after removing the cwd from sys.path.
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


In [13]:
X_train = np.array(X_train)

In [14]:
Y_train = to_categorical(Y_train)

In [15]:
#Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.25, random_state=3)

In [5]:
img_data = np.asarray( images )

In [16]:
num_classes = np.size(Y_train, 1)

## VGG-Face Model (#1)

In [67]:
image_input = Input(shape=(224,224,3))
from keras_vggface.vggface import VGGFace
vgg_face_model = VGGFace(model = 'resnet50', include_top = False, weights='vggface', input_shape=((224,224,3)))
    
vgg_face_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_14[0][0]                   
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_295 (Activation)     (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [68]:
for layer in vgg_face_model.layers[:-1]:
    layer.trainable = False
    

In [69]:
LL = vgg_face_model.get_layer('avg_pool').output
x = Flatten(name='flatten')(LL)
x = Dense(4096,name = 'fc6')(x)
x = Dropout(0.5)(x)
x = Dense(4096,name = 'fc7')(x)
x = Dropout(0.5)(x)
x = Dense(2622,name = 'fc8')(x)
out = Dense(1, activation='sigmoid',name='classifier')(x)
custom_vgg_face_model = Model(vgg_face_model.input, out)

In [70]:
custom_vgg_face_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_14[0][0]                   
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_295 (Activation)     (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [71]:
sgd = SGD(lr=0.001, decay=5e-4, momentum=0.9)
custom_vgg_face_model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [72]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 64

train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=False)

test_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=False)


train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(224,224),  
        batch_size=batch_size,
        color_mode="rgb",
        class_mode='binary')  

validation_generator = test_datagen.flow_from_directory(
        'data/validation',  # this is the target directory
        target_size=(224,224),  
        batch_size=batch_size,
        color_mode="rgb",
        class_mode='binary')


Found 22688 images belonging to 2 classes.
Found 5672 images belonging to 2 classes.


In [77]:
# fine-tune the model
filepath="vgg_face_gender_weights_improvment-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
data_size = len(image_labels.index)
train_size = data_size - (data_size * 0.2)
#tensorboard = TensorBoard(log_dir=".", histogram_freq=2000, write_graph=True, write_images=False)
callback_list = [checkpoint]

custom_vgg_face_model.fit_generator(
        train_generator,
        steps_per_epoch=train_size // batch_size,
        validation_data=validation_generator,
        validation_steps=(data_size - train_size)  // batch_size, callbacks=callback_list, epochs=75, verbose = 1)

Epoch 1/75
354/354 [==============================] - 320s 903ms/step - loss: 0.6219 - acc: 0.8532 - val_loss: 0.3157 - val_acc: 0.9123

Epoch 00001: val_acc improved from -inf to 0.91229, saving model to vgg_face_gender_weights_improvment-01-0.91.hdf5
Epoch 2/75
354/354 [==============================] - 313s 883ms/step - loss: 0.3454 - acc: 0.8812 - val_loss: 0.3294 - val_acc: 0.9119

Epoch 00002: val_acc did not improve from 0.91229
Epoch 3/75
354/354 [==============================] - 356s 1s/step - loss: 0.3110 - acc: 0.8922 - val_loss: 0.3195 - val_acc: 0.9107

Epoch 00003: val_acc did not improve from 0.91229
Epoch 4/75
354/354 [==============================] - 390s 1s/step - loss: 0.2904 - acc: 0.8998 - val_loss: 0.3178 - val_acc: 0.9132

Epoch 00004: val_acc improved from 0.91229 to 0.91317, saving model to vgg_face_gender_weights_improvment-04-0.91.hdf5
Epoch 5/75
354/354 [==============================] - 311s 878ms/step - loss: 0.2799 - acc: 0.9016 - val_loss: 0.2823 - val

KeyboardInterrupt: 

# VGG-FACE (Experiment 2)
## Attempting regularization

In [ ]:
t=time.time()

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
callback_list = [early_stop]

hist = custom_vgg_face_model.fit(X_train, y_train, batch_size=16, validation_data=(X_test, y_test), epochs=75, callbacks = callback_list, verbose=1)
#print('Training time: %s' % (t - time.time()))


In [ ]:
custom_vgg_face_model.evaluate(img_data, Y_train)

In [64]:
from keras.models import load_model
from keras.models import model_from_json
json_file = open('vgg_face_trained.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# Prediction

In [40]:
test_labels = pd.read_csv('sample_submission.csv')

In [47]:
X_testing_data = []
X_names = []
for index, row in test_labels.iterrows():
    X_names.append(row[0])
    image = imread('test/' + row[0], mode='RGB')
    image = imresize(image, (224,224,3))
    X_testing_data.append(np.array(image))
    #Y_train.append(row[1])

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [45]:
testing_data = np.asarray(X_testing_data)

In [33]:
custom_vgg_face_model.load_weights("vgg_face_gender_weights_improvment-06-0.92.hdf5")
print("Loaded model from disk")

Loaded model from disk


In [46]:
predictions = custom_vgg_face_model.predict(testing_data)

In [49]:
names = pd.DataFrame(X_names)  
gender = pd.DataFrame(predictions) 
result = pd.concat([names, gender],axis=1)
result.columns = ['Id', 'Expected']

In [50]:
print(result)

                 Id  Expected
0        test_1.jpg  0.949141
1        test_2.jpg  0.389333
2        test_3.jpg  0.973845
3        test_4.jpg  0.043681
4        test_5.jpg  0.905663
5        test_6.jpg  0.412753
6        test_7.jpg  0.963404
7        test_8.jpg  0.984018
8        test_9.jpg  0.982196
9       test_10.jpg  0.052618
10      test_11.jpg  0.004461
11      test_12.jpg  0.829718
12      test_13.jpg  0.301361
13      test_14.jpg  0.014076
14      test_15.jpg  0.684697
15      test_16.jpg  0.030375
16      test_17.jpg  0.008396
17      test_18.jpg  0.821464
18      test_19.jpg  0.892937
19      test_20.jpg  0.075250
20      test_21.jpg  0.367255
21      test_22.jpg  1.000000
22      test_23.jpg  0.997951
23      test_24.jpg  0.657232
24      test_25.jpg  0.998540
25      test_26.jpg  0.159362
26      test_27.jpg  0.005560
27      test_28.jpg  0.806311
28      test_29.jpg  0.012058
29      test_30.jpg  0.986236
...             ...       ...
7060  test_7061.jpg  0.981884
7061  test

In [51]:
result.to_csv('answers_op2_vgg_face_2.csv')